In [12]:
import os
import re
import time
import json
os.environ['GROQ_API_KEY'] = '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'
from groq import Groq
client = Groq()

conversation_history = []
filename = ''
div = '-'*500

def generate_content(prompt, history):
    history.append({
        "role": "user",
        "content": prompt
    })
    response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=history
    )
    rcontent = response.choices[0].message.content
    history.append({
        "role": "assistant",
        "content": rcontent
    })
    return rcontent

def prompt(history, filename):
    print(div)
    prompt = input("Enter your prompt: ")
    response = generate_content(prompt, history, filename)
    print(response)
    with open(filename, "a", encoding='utf-8') as f:
        f.write(div + "\n")
        f.write("User: " + prompt + "\n")
        f.write("AI: " + response + "\n")
    
def reset():
    timestamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.gmtime())
    filename = timestamp + ".txt"

    # Initialize the conversation history
    history = [
        {
            "role": "user",
            "content": "Olá, como está você?"
        },
        {
            "role": "assistant",
            "content": "Olá! Eu sou um modelo de linguagem de computador, então não tenho sentimentos nem emoções como os humanos, mas estou funcionando normalmente e pronto para ajudar com suas perguntas e tarefas! Como posso ajudá-lo hoje?"
        },
        {
            "role": "user",
            "content": "Você pode me ajudar com questões práticas de programação em C++? Cada vez que eu submeter a descrição de uma questão, quero que você gere um código solução em C++, e apenas o conteúdo do código, pronto para compilar."
        },
        {
            "role": "assistant",
            "content": "Sim, estou aqui para ajudar. Submeta a descrição da questão e eu gerarei o código solução em C++."
        }
    ]

    return filename, history

def load_history():
    with open('conversation_history.json', 'r', encoding='utf-8') as f:
        return json.load(f)
        
def write_history(history):
    with open('conversation_history.json', 'w', encoding='utf-8') as f:
        json.dump(history, f, ensure_ascii=False, indent=4)

#reset() ou load_history()
#filename=''

In [ ]:

#get-solution
startwith = ['pj','p1','p2','pu']
for tup in os.walk('./OBI'):
    pattern = re.compile(r'(\d{4})')
    src = pattern.search(tup[0])
    if src is None:
        continue
    year = src.group()
    for file in tup[2]:
        if file.endswith('.txt') and not file.endswith('-pdf.txt'):
            conversation_history = reset()[1]
            filename = year+'-'+file[:-4]+'.cpp'
            with open(os.path.join(tup[0],file), 'r', encoding='utf-8') as f:
                prompt = f.read()
            response = generate_content(prompt, conversation_history)
            with open(filename, "w", encoding='utf-8') as f:
                f.write(response.split('```')[1])
            


In [ ]:
def modify_cpp_files(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.cpp'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                
                # Find the first '#' and the last '}'
                first_hash_index = content.find('#')
                last_brace_index = content.rfind('}')
                
                if first_hash_index == -1 or last_brace_index == -1:
                    print(f"File {file} does not contain required characters.")
                    continue
                
                # Extract the content between the first '#' and the last '}'
                modified_content = content[first_hash_index:last_brace_index + 1]
                
                # Check if the content was modified
                if content != modified_content:
                    with open(file_path, 'w', encoding='utf-8') as f:
                        f.write(modified_content)
                    print(f"Modified file: {file}")

In [9]:
write_history()